Importa o modelo a ser fatiado e mapeado

In [2]:
import matplotlib.pyplot as plt
import gui
import files
from timer import Timer
from components import layer
from os import getcwd

home = getcwd()
folders = files.Paths(home)
path_input, file_name = gui.load_model(folders)
[dpi, layer_height] = gui.ask_parameters_input()
if file_name.endswith(".stl") or file_name.endswith(".STL"):
    folders.create_layers_3d(path_input, dpi, layer_height)
else:
    folders.create_layers_2d(path_input, dpi, layer_height)

layer.divide_islands(folders)

plt.rcParams["figure.figsize"]=5,5
#TODO: encontrar uma forma de jogar layerHight para o slicing W/images (POSSIVELMENTE COM O RUNSINGLEMODEL)
# TODO: retirar todos os parametros que nao uso
# TODO: dar o Typing em todos

First arg: /home/aluno/Downloads/TangranRemake/input/stl_models/CAPIVARA-BODYBUILDER.stl
Second arg: 300.0
Script executed from: /home/aluno/Downloads/TangranRemake
Script executed from: /home/aluno/Downloads/TangranRemake/input/sliced
Script executed from: /home/aluno/Downloads/TangranRemake
Script executed from: /home/aluno/Downloads/TangranRemake/slicing-with-images
number of degenerated triangles = 38
eps = 0.004
max thickness = 2
rounded plane spacing spacing = 2
model zmin = 0, model zmax = 116.52, first plane Z = -1.996, number of planes = 61
-Incremental, CAPIVARA-BODYBUILDER.stl, thickness = 2, #T = 49000, #P = 59, #S = 21164, 0.012371s (slicing), 0.019844s (polygon assembling), 0.151837s (total), *K: 0.431918


rm: não foi possível remover '*.svg': Arquivo ou diretório inexistente
rm: não foi possível remover '*.png': Arquivo ou diretório inexistente




writing image: ../input/sliced/layer_000.pgm


writing image: ../input/sliced/layer_001.pgm


writing image: ../input/sliced/layer_002.pgm


writing image: ../input/sliced/layer_003.pgm


writing image: ../input/sliced/layer_004.pgm


writing image: ../input/sliced/layer_005.pgm


writing image: ../input/sliced/layer_006.pgm


writing image: ../input/sliced/layer_007.pgm


writing image: ../input/sliced/layer_008.pgm


writing image: ../input/sliced/layer_009.pgm


writing image: ../input/sliced/layer_010.pgm


writing image: ../input/sliced/layer_011.pgm


writing image: ../input/sliced/layer_012.pgm


writing image: ../input/sliced/layer_013.pgm


writing image: ../input/sliced/layer_014.pgm


writing image: ../input/sliced/layer_015.pgm


writing image: ../input/sliced/layer_016.pgm


writing image: ../input/sliced/layer_017.pgm


writing image: ../input/sliced/layer_018.pgm


writing image: ../input/sliced/layer_019.pgm


writing image: ../input/sliced/layer_020.pgm


writing ima

Paredes Finas

In [4]:
import matplotlib.pyplot as plt
import gui
import files
from timer import Timer
from components import layer
from os import getcwd
try:
    folders
except NameError:
    home = getcwd()
    folders = files.Paths(home)

[nozzle_diam_internal,nozzle_diam_external] = gui.ask_parameters_thin_walls()
# from components.layer import Layer
camadas = folders.list(layers=1)
for i,c in enumerate(camadas):
    camada = folders.load_layer_json(c)
    with Timer("Fase 1: Camada: " + str(i)):
        camada.make_thin_walls(folders, nozzle_diam_external, nozzle_diam_internal)
    
# TODO: arvore de paredes finas
# TODO: quando duas áreas se encontram
# TODO: limite de parede diam minimo
# TODO: anotar todos os (não fechou contorno)
# TODO arrumar para origens extremamente pequenas, ignorar da problema
# TODO arrumar casos onde a beirada esta vazndo criando areas triangulares distantes
# FIXME: Precisa mesmo separar linha 1 2 topo e baixo? muitos elementos estão sendo criados

Fase 1: Camada: 0
Criando paredes finas
Retirando Paredes finas da camada
Total de [Retirando Paredes finas da camada] : 0.1310257911682129
Total de [Criando paredes finas] : 1.9049274921417236
Total de [Fase 1: Camada: 0] : 1.905712366104126
Fase 1: Camada: 1
Criando paredes finas
Retirando Paredes finas da camada
Total de [Retirando Paredes finas da camada] : 0.08891558647155762
Total de [Criando paredes finas] : 1.8061316013336182
Total de [Fase 1: Camada: 1] : 1.8062608242034912
Fase 1: Camada: 2
Criando paredes finas
Retirando Paredes finas da camada
Total de [Retirando Paredes finas da camada] : 0.0917367935180664
Total de [Criando paredes finas] : 1.6976866722106934
Total de [Fase 1: Camada: 2] : 1.6978132724761963
Fase 1: Camada: 3
Criando paredes finas
teste: caso de uma unica linha no entorno da origem
OK: fechou contorno
Value(False)
Retirando Paredes finas da camada
Total de [Retirando Paredes finas da camada] : 0.15877509117126465
Total de [Criando paredes finas] : 25.2805

[ WARN:185@364.245] global loadsave.cpp:248 findDecoder imread_(''): can't open/read file: check file path/integrity


Contornos

In [ ]:
import matplotlib.pyplot as plt
import gui
import files
from timer import Timer
from components import layer
from os import getcwd
try:
    folders
except NameError:
    home = getcwd()
    folders = files.Paths(home)

[void_max,internal_max,external_max] = gui.ask_parameters_offsets()
# from components.layer import Layer
camadas = folders.list(layers=1)
for i,c in enumerate(camadas):
    camada = folders.load_layer_json(c)
    with Timer("Fase 1: Camada: " + str(i)):
        camada.make_offsets(folders, void_max, external_max, internal_max)

Fase 1: Camada: 0
Criando Lvls
Ilha: 0 Número de Níveis: 7
Total de [Criando Lvls] : 7.338462829589844
Criendo os loops
Total de [Criendo os loops] : 2.2626430988311768
Criando regiões de influência
Total de [Criando regiões de influência] : 0.00023436546325683594
Total de [Fase 1: Camada: 0] : 1497.060544013977


AttributeError: 'OffsetRegions' object has no attribute 'create_influence_regions'